In [3]:
import pandas as pd
import gzip
import io
import os

### KPI 1: Mejorar un 10% la accesibilidad de los restaurantes en Florida 

Criterio: Según el ratio de la relación de cantidad de restaurantes sobre cantidad de habitantes por estado, Florida es del menor ratio.

count business_id / habitantes 

KPIs:

Enfoque técnico: Optimización del alcance y calidad del servicio        
Mejorar un 10% la accesibilidad de los restaurantes en Florida      
    Cantidad de restaurantes por estado cada 1000 habitantes        
    business_google.json.gz         

Falta crear tabla auxiliar de cantidad de habitantes por estado por año

Evaluar calidad media de productos por categoría de restaurantes
Recomendación según top 20 categorias, según correlación entre categoria y stars a través del tiempo            
    Correlación entre las estrellas y las 20 categorías con mayor frecuencia        
    business_yelp.json.gz           


Enfoque orgánico: Networking            
Mejorar la tasa de diferencia entre reviews y respuestas 
"Aumentar la calidad de la comunicación a través de un mejor ratio respuestas por comentarios y  disminuir el tiempo promedio de respuestas"

kpi = (ratio respuestas/review * rango tiempo) + sentiment promedio

Datos disponibles: respuesta/review, tiempo promedio y sentiment promedio
    Ratio: respuestas / reviews, respuestas tiempo promedio
    business_id, resp time != 0, sentiment_score !=0, date             
    reviews_google.parquet.gz           

Promover la tasa de retención de cliente            
    Cantidad de meses distintos con review para usuario         
    reviews_google.parquet.gz           

Aumentar 10% reviews de usuarios poco influyentes y 1% muy influyente            
    Cantidad de usuarios inluyentes y poco influyentes         
    users_yelp.parquet.gz 
    

    
metricas        
ratio useful/review-count       

esperar dataset de dami     

Sistema de recomendación para empresas: Análisis de ubicación       
    Por cantidad competencia en la zona     
    Por calidad competencia en la zona      
    Por horario por atributo            
Sistema de recomendación para empresas: Publicidad      
    Promoción por gustos del usuario        
    Por gustos de usuarios similares        
    Por gustos de amigos del usuario        
